In [2]:
%load_ext autoreload

%autoreload 2

import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
matplotlib.rcParams["image.composite_image"]=False
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import scipy.linalg
from numpy.fft import rfft, irfft, fft, ifft

import multiprocessing
import concurrent.futures
c

import psrchive
import residuals

import pipe
from pipe import rotate_phase, convert_template, align_profile, align_scale_profile

import template_match as tm

plt.viridis()

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
obs = "data/obs/56494.49_GBT_1400"
processing_name = "onescintle"
scr, = glob(join(obs,processing_name,"scrunch_*.ar"))
F = psrchive.Archive_load(scr)
hdulist = fits.open(scr)

In [4]:
hdulist.info()

Filename: data/obs/56494.49_GBT_1400/onescintle/scrunch_0000.ar
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU      64   ()      
  1  HISTORY     BinTableHDU     71   8R x 28C   [24A, 256A, 8A, 8A, 1J, 1I, 1I, 1I, 1D, 1D, 1J, 1D, 1D, 1D, 1I, 1I, 1I, 1I, 1I, 32A, 32A, 32A, 256A, 32A, 32A, 1I, 32A, 1I]   
  2  PSRPARAM    BinTableHDU     12   38R x 1C   [128A]   
  3  POLYCO      BinTableHDU     38   1R x 13C   [24A, 16A, 1I, 1I, 1I, 8A, 1D, 1D, 1D, 1D, 1D, 1D, 15D]   
  4  SUBINT      BinTableHDU     88   2R x 20C   [1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1E, 1E, 1E, 1E, 1E, 1D, 1D, 128D, 128E, 512E, 512E, 524288I]   


In [6]:
hdulist['SUBINT'].data.dtype

dtype((numpy.record, [('INDEXVAL', '>f8'), ('TSUBINT', '>f8'), ('OFFS_SUB', '>f8'), ('LST_SUB', '>f8'), ('RA_SUB', '>f8'), ('DEC_SUB', '>f8'), ('GLON_SUB', '>f8'), ('GLAT_SUB', '>f8'), ('FD_ANG', '>f4'), ('POS_ANG', '>f4'), ('PAR_ANG', '>f4'), ('TEL_AZ', '>f4'), ('TEL_ZEN', '>f4'), ('AUX_DM', '>f8'), ('AUX_RM', '>f8'), ('DAT_FREQ', '>f8', (128,)), ('DAT_WTS', '>f4', (128,)), ('DAT_OFFS', '>f4', (512,)), ('DAT_SCL', '>f4', (512,)), ('DATA', '>i2', (4, 128, 1024))]))

In [7]:
hdulist['SUBINT'].columns

ColDefs(
    name = 'INDEXVAL'; format = '1D'
    name = 'TSUBINT'; format = '1D'; unit = 's'
    name = 'OFFS_SUB'; format = '1D'; unit = 's'
    name = 'LST_SUB'; format = '1D'; unit = 's'
    name = 'RA_SUB'; format = '1D'; unit = 'deg'
    name = 'DEC_SUB'; format = '1D'; unit = 'deg'
    name = 'GLON_SUB'; format = '1D'; unit = 'deg'
    name = 'GLAT_SUB'; format = '1D'; unit = 'deg'
    name = 'FD_ANG'; format = '1E'; unit = 'deg'
    name = 'POS_ANG'; format = '1E'; unit = 'deg'
    name = 'PAR_ANG'; format = '1E'; unit = 'deg'
    name = 'TEL_AZ'; format = '1E'; unit = 'deg'
    name = 'TEL_ZEN'; format = '1E'; unit = 'deg'
    name = 'AUX_DM'; format = '1D'; unit = 'CM-3'
    name = 'AUX_RM'; format = '1D'; unit = 'RAD'
    name = 'DAT_FREQ'; format = '128D'; unit = 'MHz'
    name = 'DAT_WTS'; format = '128E'
    name = 'DAT_OFFS'; format = '512E'
    name = 'DAT_SCL'; format = '512E'
    name = 'DATA'; format = '524288I'; unit = 'Jy'; dim = '(1024,128,4)'
)

In [13]:
hdulist['SUBINT'].data['DATA'].shape

(2, 4, 128, 1024)

In [14]:
hdulist['SUBINT'].data['DAT_WTS'].shape

(2, 128)

In [15]:
hdulist['SUBINT'].data['DAT_FREQ'].shape

(2, 128)

In [16]:
hdulist['SUBINT'].header

XTENSION= 'BINTABLE'           / ***** Subintegration data  *****               
BITPIX  =                    8 / N/A                                            
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =              1054308 / width of table in bytes                        
NAXIS2  =                    2 / Number of rows in table (NSUBINT)              
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                   20 / Number of fields per row                       
TTYPE1  = 'INDEXVAL'           / Optionally used if INT_TYPE != TIME            
TFORM1  = '1D      '           / Double                                         
TTYPE2  = 'TSUBINT '           / Length of subintegration                       
TFORM2  = '1D      '           / Double                                         
TTYPE3  = 'OFFS_SUB'        

In [17]:
hdulist['SUBINT'].data['OFFS_SUB']

array([ 326.4169081 ,  981.02133036])

In [19]:
hdulist[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
COMMENT   FITS (Flexible Image Transport System) format defined in Astronomy and
COMMENT   Astrophysics Supplement Series v44/p363, v44/p371, v73/p359, v73/p365.
COMMENT   Contact the NASA Science Office of Standards and Technology for the   
COMMENT   FITS Definition document #100 and other FITS information.             
HDRVER  = '5.4             '   / Header version                                 
FITSTYPE= 'PSRFITS         '   / FITS definition for pulsar data files          
DATE    = '2017-04-19T09:52:

In [22]:
d_p = F.get_data()
d_f = hdulist['SUBINT'].data['DATA']

np.mean(d_p-d_f)

2531.1475

In [23]:
F.get_state()

'Coherence'

In [24]:
hdulist['SUBINT'].data['INDEXVAL']

array([ 0.,  0.])

In [26]:
hdulist['HISTORY'].columns

ColDefs(
    name = 'DATE_PRO'; format = '24A'
    name = 'PROC_CMD'; format = '256A'
    name = 'SCALE'; format = '8A'
    name = 'POL_TYPE'; format = '8A'
    name = 'NSUB'; format = '1J'
    name = 'NPOL'; format = '1I'
    name = 'NBIN'; format = '1I'
    name = 'NBIN_PRD'; format = '1I'
    name = 'TBIN'; format = '1D'; unit = 's'
    name = 'CTR_FREQ'; format = '1D'; unit = 'MHz'
    name = 'NCHAN'; format = '1J'
    name = 'CHAN_BW'; format = '1D'; unit = 'MHz'
    name = 'DM'; format = '1D'; unit = 'CM-3'
    name = 'RM'; format = '1D'; unit = 'RAD'
    name = 'PR_CORR'; format = '1I'
    name = 'FD_CORR'; format = '1I'
    name = 'BE_CORR'; format = '1I'
    name = 'RM_CORR'; format = '1I'
    name = 'DEDISP'; format = '1I'
    name = 'DDS_MTHD'; format = '32A'
    name = 'SC_MTHD'; format = '32A'
    name = 'CAL_MTHD'; format = '32A'
    name = 'CAL_FILE'; format = '256A'
    name = 'RFI_MTHD'; format = '32A'
    name = 'RM_MODEL'; format = '32A'
    name = 'AUX_RM_C'; format